In [1]:
import os
import sys
import re
current_folder = globals()['_dh'][0]
project_path = re.sub(r'TeNNetPy.*', 'TeNNetPy/', str(current_folder))
os.chdir(project_path)
sys.path.append(project_path)

In [ ]:
import pandas as pd
import numpy as np
from Utils.db_utils import read_sql_query

In [61]:
query = """SELECT b.*,
                            tourney_name,
                            tourney_level,
                            winner_name,
                            loser_name,
                            round,
                            surface,
                            match_settled,
                            tourney_date,
                            winner_pred,
                            loser_pred,
                            'doubles' = TRUE as doubles,
                            'atp' as compet
                                    FROM Bet b join men_matchs m on (b.ID_MATCH = m.ID_MATCH) 
                                        right join predictions p on (m.ID_MATCH = p.ID_MATCH)
                                        WHERE match_settled in (1,2) and score != 'W/O'
                    UNION
                        SELECT b.*,
                            tourney_name,
                            tourney_level,
                            winner_name,
                            loser_name,
                            round,
                            surface,
                            match_settled,
                            tourney_date,
                            winner_pred,
                            loser_pred,
                            'doubles' = TRUE as doubles,
                            'wta' as compet
                                    FROM  Bet b join women_matchs m on (b.ID_MATCH = m.ID_MATCH)
                                        right join predictions p on (m.ID_MATCH = p.ID_MATCH)
                                        WHERE  match_settled in (1,2) and score != 'W/O'
                    UNION
                        SELECT b.*, 
                                tourney_name,
                                tourney_level,
                                concat(winner_name1,'/',winner_name2) as winner_name,
                                concat(loser_name1,'/',loser_name2)  as loser_name,
                                round,
                                surface,
                                match_settled, 
                                tourney_date, 
                                winner_pred,
                                loser_pred,
                                'doubles' = FALSE as doubles  ,
                                'doubles' as compet
                                    FROM Bet b join double_matchs m  on (b.ID_MATCH = m.ID_MATCH) 
                                        right join predictions p on (m.ID_MATCH = p.ID_MATCH)
                                        WHERE match_settled in (1,2) and score != 'W/O'"""

In [62]:

schema = "TeNNet"
bets = read_sql_query(schema=schema, query=query)
bets = bets[bets["ID_USER"] == 1]
bets.sort_values(by="tourney_date", ascending=True, inplace=True)
bets.reset_index(drop=True, inplace=True)

2026-01-06 22:26:58,862-INFO: Reading table from schema TeNNet with query: SELECT b.*,
                            tourney_name,
                            tourney_level,
                            winner_name,
                            loser_name,
                            round,
                            surface,
                            match_settled,
                            tourney_date,
                            winner_pred,
                            loser_pred,
                            'doubles' = TRUE as doubles,
                            'atp' as compet
                                    FROM Bet b join men_matchs m on (b.ID_MATCH = m.ID_MATCH) 
                                        right join predictions p on (m.ID_MATCH = p.ID_MATCH)
                                        WHERE match_settled in (1,2) and score != 'W/O'
                    UNION
                        SELECT b.*,
                            tourney_name,
                           

In [68]:
bets["Match"] = bets["winner_name"] + " - " + bets["loser_name"]
bets["real_odds"] = (1 / (bets["odds"] - 1)) * 0.97 + 1
bets["cote_pred"] = np.where(
    (bets["match_settled"] == 1) & (bets["bet"] == 1)
    | (bets["match_settled"] == 2) & (bets["bet"] == 0),
    bets["winner_pred"],
    bets["loser_pred"],
)
bets["player_bet"] = np.where(bets["bet"] == 1, bets["winner_name"], bets["loser_name"])
bets["win"] = np.where(
    (bets["match_settled"] == 1) & (bets["bet"] == 1)
    | (bets["match_settled"] == 2) & (bets["bet"] == 0),
    1,
    0,
)
bets["net_gain"] = np.where(
    bets["win"] == 1, bets["real_odds"] * bets["stake"] - bets["stake"], -bets["stake"]
)
bets["net_unit"] = bets["net_gain"] / bets["stake"]
bets["marge_unit"] = bets["real_odds"] / bets["cote_pred"] - 1
bets["marge"] = bets["marge_unit"] * bets["stake"]

In [54]:
bets

,ID_BET,ID_USER,ID_MATCH,ID_MARKET,ID_REF,type,bet,bet_libelle,odds,stake,...,compet,Match,real_odds,cote_pred,player_bet,win,net_gain,net_unit,marge_unit,marge
0,11,1,YgInik7c,1.252157018,208405757,Match Odds,0,Maria Sakkari,2.74,107.86,...,wta,Osaka N. - Sakkari M.,1.557471,2.91498,Sakkari M.,1,60.128851,0.557471,-0.465701,-50.230496
1,9,1,YgInik7c,1.252157018,208405667,Match Odds,0,Maria Sakkari,2.72,17.44,...,wta,Osaka N. - Sakkari M.,1.563953,2.91498,Sakkari M.,1,9.835349,0.563953,-0.463477,-8.083041
2,3,1,SlXlWGl4,1.252243638,208395907,Match Odds,0,Antonia Ruzic,1.44,1.14,...,wta,Timofeeva M. - Ruzic A.,3.204545,3.16566,Ruzic A.,1,2.513182,2.204545,0.012284,0.014003
3,2,1,zyCXtY2M,1.252243658,208395872,Match Odds,1,Anhelina Kalinina,1.35,11.88,...,wta,Bondar A. - Kalinina A.,3.771429,3.71878,Bondar A.,1,32.924571,2.771429,0.014157,0.168191
4,4,1,xOYRyD2n,1.252156562,208397585,Match Odds,1,Sebastian Baez,4.80,183.20,...,atp,Baez S. - Munar J.,1.255263,1.20909,Baez S.,0,-183.200000,-1.000000,0.038188,6.996107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,428,1,EVpUQZIs,1.252401037,208774547,Match Odds,0,Tiago Pereira,3.50,29.53,...,atp,Ritschard A. - Pereira T.,1.388000,1.34407,Pereira T.,1,11.457640,0.388000,0.032684,0.965168
232,430,1,rup0U6a3,1.252332942,208781690,Match Odds,1,Tyler Zink,3.20,46.99,...,atp,Kozlov S. - Zink T.,1.440909,1.40577,Kozlov S.,1,20.718318,0.440909,0.024996,1.174578
233,432,1,4fY8SpUF,1.252332952,208784351,Match Odds,1,Tom Gentzsch,2.78,10.54,...,atp,Gentzsch T. - Brunold M.,1.544944,1.51851,Gentzsch T.,0,-10.540000,-1.000000,0.017408,0.183478
234,434,1,YwrEt1Vq,1.252321052,208787429,Match Odds,1,Duncan/Mackinlay,1.75,5.66,...,doubles,Duncan S./MacKinlay J. - Murgett F./Story J.,2.293333,2.17765,Duncan S./MacKinlay J.,0,-5.660000,-1.000000,0.053123,0.300676


In [38]:
bets.columns

Index(['ID_BET', 'ID_USER', 'ID_MATCH', 'ID_MARKET', 'ID_REF', 'type', 'bet',
       'bet_libelle', 'odds', 'stake', 'status', 'tourney_name',
       'tourney_level', 'winner_name', 'loser_name', 'round', 'match_settled',
       'tourney_date', 'winner_pred', 'loser_pred', 'doubles', 'compet',
       'Match', 'real_odds', 'cote_pred', 'player_bet', 'win', 'net_gain',
       'net_unit', 'marge_unit', 'marge'],
      dtype='object')

In [74]:
prepared_bets = bets[["ID_MATCH",'Match','tourney_date', 'compet','surface','tourney_level',
       'player_bet', 'stake', 'round', 
       'real_odds', 'cote_pred', 'net_gain','marge']].copy()
prepared_bets["real_odds"] = prepared_bets["real_odds"].round(3)
prepared_bets["cote_pred"] = prepared_bets["cote_pred"].round(3)
prepared_bets["marge"] = prepared_bets["marge"].round(2)
prepared_bets["stake"] = prepared_bets["stake"].round(2)
prepared_bets["net_gain"] = prepared_bets["net_gain"].round(2)
prepared_bets.rename(columns={'tourney_date':'Date', 'tourney_level':'Level','compet':'Compétition','surface':'Surface',
                            'stake':'Mise', 'round':'Round',
                            'real_odds':'Cote', 'cote_pred':'Prédiction',
                            'net_gain':'Gains net', 'marge':'Marge attendue'}, inplace=True)
prepared_bets

,ID_MATCH,Match,Date,Compétition,Surface,Level,player_bet,Mise,Round,Cote,Prédiction,Gains net,Marge attendue
0,YgInik7c,Osaka N. - Sakkari M.,2025-12-02 10:00:00,wta,Hard,I,Sakkari M.,107.86,R64,1.557,2.915,60.13,-50.23
1,YgInik7c,Osaka N. - Sakkari M.,2025-12-02 10:00:00,wta,Hard,I,Sakkari M.,17.44,R64,1.564,2.915,9.84,-8.08
2,SlXlWGl4,Timofeeva M. - Ruzic A.,2026-01-02 01:10:00,wta,Hard,P,Ruzic A.,1.14,Q2,3.205,3.166,2.51,0.01
3,zyCXtY2M,Bondar A. - Kalinina A.,2026-01-02 01:10:00,wta,Hard,P,Bondar A.,11.88,Q2,3.771,3.719,32.92,0.17
4,xOYRyD2n,Baez S. - Munar J.,2026-01-02 03:15:00,atp,Hard,A,Baez S.,183.20,R64,1.255,1.209,-183.20,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,EVpUQZIs,Ritschard A. - Pereira T.,2026-01-06 14:15:00,atp,Hard,C,Pereira T.,29.53,R32,1.388,1.344,11.46,0.97
232,rup0U6a3,Kozlov S. - Zink T.,2026-01-06 15:45:00,atp,Hard,C,Kozlov S.,46.99,R32,1.441,1.406,20.72,1.17
233,4fY8SpUF,Gentzsch T. - Brunold M.,2026-01-06 16:40:00,atp,Hard,C,Gentzsch T.,10.54,R32,1.545,1.519,-10.54,0.18
234,YwrEt1Vq,Duncan S./MacKinlay J. - Murgett F./Story J.,2026-01-06 17:10:00,doubles,Hard,C,Duncan S./MacKinlay J.,5.66,R16,2.293,2.178,-5.66,0.30


In [78]:
#group by Match and player_bet je voudrais ponderer les cotes par la mise totale
grouped_bets = prepared_bets.groupby(['ID_MATCH','Match','player_bet']).agg(
    {'Date':'first',
     'Compétition':'first',
     'Level':'first',
     'Round':'first',
     'Surface':'first',
     'Mise':'sum',
     'Cote': lambda x: np.average(x, weights=prepared_bets.loc[x.index, 'Mise']),
     'Prédiction': 'mean',
     'Gains net':'sum',
     'Marge attendue':'sum'
    }
).reset_index()
grouped_bets['Cote'] = grouped_bets['Cote'].round(3)
grouped_bets['Prédiction'] = grouped_bets['Prédiction'].round(3)
grouped_bets['Marge attendue'] = grouped_bets['Marge attendue'].round(2)
grouped_bets.sort_values(by='Date', ascending=True, inplace=True)
grouped_bets

,ID_MATCH,Match,player_bet,Date,Compétition,Level,Round,Surface,Mise,Cote,Prédiction,Gains net,Marge attendue
80,YgInik7c,Osaka N. - Sakkari M.,Sakkari M.,2025-12-02 10:00:00,wta,I,R64,Hard,125.30,1.558,2.915,69.97,-58.31
65,SlXlWGl4,Timofeeva M. - Ruzic A.,Ruzic A.,2026-01-02 01:10:00,wta,P,Q2,Hard,1.14,3.205,3.166,2.51,0.01
145,zyCXtY2M,Bondar A. - Kalinina A.,Bondar A.,2026-01-02 01:10:00,wta,P,Q2,Hard,11.88,3.771,3.719,32.92,0.17
135,xOYRyD2n,Baez S. - Munar J.,Baez S.,2026-01-02 03:15:00,atp,A,R64,Hard,183.20,1.255,1.209,-183.20,7.00
42,Ioz0ToFj,Sierra S. - Bouzas Maneiro J.,Sierra S.,2026-01-02 05:20:00,wta,I,R64,Hard,138.39,1.380,1.349,-138.39,3.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,EVpUQZIs,Ritschard A. - Pereira T.,Pereira T.,2026-01-06 14:15:00,atp,C,R32,Hard,192.00,1.379,1.344,72.87,5.07
127,rup0U6a3,Kozlov S. - Zink T.,Kozlov S.,2026-01-06 15:45:00,atp,C,R32,Hard,46.99,1.441,1.406,20.72,1.17
11,4fY8SpUF,Gentzsch T. - Brunold M.,Gentzsch T.,2026-01-06 16:40:00,atp,C,R32,Hard,10.54,1.545,1.519,-10.54,0.18
85,YwrEt1Vq,Duncan S./MacKinlay J. - Murgett F./Story J.,Duncan S./MacKinlay J.,2026-01-06 17:10:00,doubles,C,R16,Hard,5.66,2.293,2.178,-5.66,0.30
